# ES → BBF Salesforce Off-Net Migration

This notebook migrates Off_Net__c records from ES Salesforce to BBF Salesforce.

## Prerequisites
- **Service__c migration must be completed first** (ES Orders have BBF_New_Id__c populated)
- ES Off_Net__c records must have BBF_New_Id__c field created

## Object Mapping
- **ES Source:** Off_Net__c (Custom Object)
- **BBF Target:** Off_Net__c (Custom Object)

## Key Relationship
- **ES:** `Off_Net__c.SOF1__c` → Order (NOT Implementation__c which is deprecated!)
- **BBF:** `Off_Net__c.Service__c` → Service__c (optional lookup)
- **Mapping:** ES Order.BBF_New_Id__c = BBF Service__c.Id

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query Off_Net__c from ES where:
   - SOF1__c (Order) has BBF_New_Id__c populated (Service already migrated)
   - Off_Net__c does NOT have BBF_New_Id__c populated (not yet migrated)
3. Transform ES Off_Net__c for BBF Off_Net__c schema:
   - Set OwnerId (ONLY required field)
   - Set Service__c = ES Order.BBF_New_Id__c (optional)
   - Add ES_Legacy_ID__c = ES Off_Net.Id (for tracking)
4. Insert Off_Net__c to BBF Salesforce
5. Update ES Off_Net__c with BBF_New_Id__c = BBF Off_Net.Id
6. Output results to Excel

## BBF Off_Net__c Required Fields
| Field | Type | Notes |
|-------|------|-------|
| Name | Autonumber | Auto-generated, don't set |
| OwnerId | Reference | **ONLY required field** |

## Field Tracking Strategy
**In BBF Off_Net__c:** `ES_Legacy_ID__c` stores original ES Off_Net ID

**In ES Off_Net__c:** `BBF_New_Id__c` stores new BBF Off_Net ID after migration

In [ ]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL Off-Net records
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Off-Net Owner - Set all migrated Off-Net records to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as other migrations

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_offnet_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

In [ ]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

In [ ]:
# === QUERY ES OFF_NET__C ===
# Day 1: MINIMAL FIELDS ONLY
# BBF Off_Net__c: Name is AUTONUMBER, OwnerId is ONLY required field
# ES relationship: SOF1__c links to Order (NOT Implementation__c which is deprecated)

print("\n" + "=" * 80)
print("QUERYING ES OFF_NET__C - MINIMAL FIELDS")
print("=" * 80)

# MINIMAL QUERY:
# - Id: for ES_Legacy_ID__c tracking
# - SOF1__r.BBF_New_Id__c: Order's BBF Service ID for optional Service__c lookup
# NOTE: SOF1__c links to Order, Order.BBF_New_Id__c = BBF Service__c.Id
query = """SELECT Id, Name, SOF1__c, SOF1__r.BBF_New_Id__c, SOF1__r.OrderNumber
FROM Off_Net__c
WHERE SOF1__c != null
AND SOF1__r.BBF_New_Id__c != null
AND SOF1__r.BBF_New_Id__c != ''
AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')"""

if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query:\n{query}")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_offnets = result["records"]

print(f"✅ Found {len(es_offnets)} Off_Net__c records to migrate")

if len(es_offnets) > 0:
    sample = es_offnets[0]
    sof = sample.get("SOF1__r", {}) or {}
    print(f"\nSample Off_Net__c:")
    print(f"  ID:              {sample['Id']}")
    print(f"  Name:            {sample.get('Name', 'N/A')}")
    print(f"  Order (SOF1):    {sof.get('OrderNumber', 'N/A')}")
    print(f"  BBF Service ID:  {sof.get('BBF_New_Id__c', 'N/A')}")
else:
    print("\n✅ No Off_Net__c records to migrate (or all already migrated)")

In [ ]:
# === TRANSFORM FOR BBF OFF_NET__C ===
# Day 1 Migration: REQUIRED FIELDS ONLY
# - Name: AUTONUMBER (don't set)
# - OwnerId: ONLY required field
# - Service__c: Optional lookup (from SOF1__r.BBF_New_Id__c)

print("\n" + "=" * 80)
print("TRANSFORMING OFF_NET__C FOR BBF (REQUIRED FIELDS ONLY)")
print("=" * 80)

bbf_offnets = []
skipped_records = []

for es_offnet in es_offnets:
    # Get BBF Service ID from the Order relationship
    bbf_service_id = None
    sof = es_offnet.get("SOF1__r") or {}
    if sof.get("BBF_New_Id__c"):
        bbf_service_id = sof["BBF_New_Id__c"]

    # =========================================================================
    # BBF Off_Net__c - REQUIRED FIELDS ONLY
    # Name = Autonumber (don't set)
    # OwnerId = ONLY required field
    # Service__c = Optional lookup
    # =========================================================================
    bbf_offnet = {
        # 🔴 REQUIRED: OwnerId (only required field)
        "OwnerId": OWNER_ID,
        # 🔗 Tracking
        "ES_Legacy_ID__c": es_offnet["Id"],
    }

    # Optional: Service lookup (from Order.BBF_New_Id__c which IS the BBF Service ID)
    if bbf_service_id:
        bbf_offnet["Service__c"] = bbf_service_id

    bbf_offnets.append(bbf_offnet)

print(f"✅ Transformed {len(bbf_offnets)} Off-Net records")
print(f"\n   REQUIRED FIELDS SET:")
print(f"   - OwnerId = {OWNER_ID} (ONLY required field)")
print(f"   - ES_Legacy_ID__c (tracking)")
print(f"   - Service__c (optional, from Order.BBF_New_Id__c)")
print(f"\n   Note: Name is Autonumber (auto-generated)")

In [ ]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING OFF-NET RECORDS TO BBF")
print("=" * 80)

if len(bbf_offnets) == 0:
    print("⚠️  No Off-Net records to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_offnets)} Off-Net records using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Off_Net__c.insert(bbf_offnets)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_offnets[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "bbf_service_id": bbf_offnets[i].get("Service__c", ""),
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_offnets[i]["ES_Legacy_ID__c"],
                        "errors": result["errors"],
                        "bbf_service_id": bbf_offnets[i].get("Service__c", ""),
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} Off-Net records")
        print(f"❌ Failed to insert: {len(failed_inserts)} Off-Net records")

        if len(failed_inserts) > 0:
            print(f"\nFailed Off-Net records (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - ES ID: {item['es_id']}")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES Off_Net ID:   {sample['es_id']}")
            print(f"  BBF Off_Net ID:  {sample['bbf_id']}")
            print(f"  BBF Service ID:  {sample['bbf_service_id'] or 'N/A'}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []

In [ ]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No Off-Net records to update in ES")
    es_update_results = []
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} Off_Net__c records in ES...")
    print("   - Setting BBF_New_Id__c = BBF Off_Net ID")

    try:
        es_update_results = es_sf.bulk.Off_Net__c.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(f"\n✅ Successfully updated: {success_count} Off_Net__c records in ES")
        print(f"❌ Failed to update: {error_count} Off_Net__c records in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []

In [ ]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

status_colors = {
    "Success": "C6EFCE",
    "Failed": "FFC7CE",
}

# --- SHEET 1: Migration Results ---
headers1 = ["ES Off_Net ID", "BBF Off_Net ID", "BBF Service ID", "Status", "Error"]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "BBF_Service_ID": item["bbf_service_id"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "BBF_Service_ID": item["bbf_service_id"],
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append([r["ES_ID"], r["BBF_ID"], r["BBF_Service_ID"], r["Status"], r["Error"]])
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Off-Net Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A7"].font = Font(bold=True)
ws2["B7"].font = Font(bold=True)
ws2.append(["Total Off-Net Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Off_Net ID", "BBF Off_Net ID", "BBF Service ID"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["bbf_service_id"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Off_Net ID", "BBF Service ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append([item["es_id"], item["bbf_service_id"], str(item["errors"])])

for col in ws4.columns:
    col_cells = list(col)
    max_length = max(len(str(cell.value)) for cell in col_cells) if col_cells else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} Off-Net records)")
print(f"   📈 Sheet 2: Summary")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} mappings)")
print(f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts)} failures)")

In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Off_Net__c queried: {len(es_offnets)}")
print(f"BBF Off_Net__c inserted: {len(successful_inserts)}")
if len(es_offnets) > 0:
    print(f"Success rate: {len(successful_inserts)/len(es_offnets)*100:.1f}%")
else:
    print("Success rate: N/A - No Off-Net records to process")
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} Off-Net records.")
    print("   To migrate ALL, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Off-Net migration finished.")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} Off-Net records failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")

---
## Day 1 Migration Summary

### Required Fields Set
| BBF Field | Value | Notes |
|-----------|-------|-------|
| OwnerId | OWNER_ID config | **ONLY required field** |
| ES_Legacy_ID__c | ES Off_Net.Id | Tracking |
| Service__c | From SOF1__r.BBF_New_Id__c | Optional lookup |

### Fields NOT Set (Why)
| Field | Reason |
|-------|--------|
| Name | Autonumber - auto-generated |

### ES to BBF Relationship Mapping
| ES Field | ES Object | BBF Field | Notes |
|----------|-----------|-----------|-------|
| SOF1__c | Order | Service__c | Order.BBF_New_Id__c = BBF Service ID |
| Location_1__c | Address__c | AA_Location__c | Day 2 enrichment |
| Location_2__c | Address__c | ZZ_Location__c | Day 2 enrichment |

**Important:** The ES `Implementation__c` field is DEPRECATED and links to `IMPLEMENTATION_Project__c`, NOT Orders. Use `SOF1__c` which correctly links to the Order object.

---
## Cleanup Apex (if needed)

### Delete Migrated Off-Net from BBF
```apex
List<Off_Net__c> offnets = [SELECT Id FROM Off_Net__c WHERE ES_Legacy_ID__c != null];
System.debug('Found ' + offnets.size() + ' migrated Off-Net records');
delete offnets;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<Off_Net__c> offnets = [SELECT Id, BBF_New_Id__c 
                           FROM Off_Net__c 
                           WHERE BBF_New_Id__c != NULL];
System.debug('Found ' + offnets.size() + ' records to reset');
for (Off_Net__c offnet : offnets) {
    offnet.BBF_New_Id__c = NULL;
}
update offnets;
```